In [113]:
import numpy as np
import pandas as pd


#### some function

In [114]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))


def relu(Z):
    return np.maximum(0, Z)


def sigmoid_back(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)


def relu_backward(dA, Z):
    # print(dA.shape)
    # print("z", Z.shape)
    # print(Z <= 0)
    dZ = np.array(dA, copy=True)
    # print(dZ)
    dZ[Z <= 0] = 0

    # print("1")
    return dZ


#### DATA process

In [115]:
train_df = pd.read_csv("3.csv")
train_df = train_df.drop(['PassengerId'], axis=1)
print(train_df.head)
idx = np.random.permutation(train_df.shape[0])
# print(idx)
train_df = train_df.iloc[idx]
test_df, train_df = train_df[0: 200], train_df[200:]


<bound method NDFrame.head of      Survived  Pclass  Sex       Age  SibSp     Parch      Fare  Embarked
0         0.0     1.0  0.0  0.271174  0.125  0.000000  0.014151       0.0
1         1.0     0.0  1.0  0.472229  0.125  0.000000  0.139136       0.5
2         1.0     1.0  1.0  0.321438  0.000  0.000000  0.015469       0.0
3         1.0     0.0  1.0  0.434531  0.125  0.000000  0.103644       0.0
4         0.0     1.0  0.0  0.434531  0.000  0.000000  0.015713       0.0
..        ...     ...  ...       ...    ...       ...       ...       ...
886       0.0     0.5  0.0  0.334004  0.000  0.000000  0.025374       0.0
887       1.0     0.0  1.0  0.233476  0.000  0.000000  0.058556       0.0
888       0.0     1.0  1.0  0.336517  0.125  0.333333  0.045771       0.0
889       1.0     0.0  0.0  0.321438  0.000  0.000000  0.058556       0.5
890       0.0     1.0  0.0  0.396833  0.000  0.000000  0.015127       1.0

[891 rows x 8 columns]>


#### model

In [116]:
class model():
    def __init__(self, input_dim, learning_rate):
        super(model, self).__init__()
        self.input_dim = input_dim
        self.learning_rate = learning_rate
        self.size_l = [(input_dim, 8), (8, 4), (4, 2), (2, 1)]
        self.deep = len(self.size_l)
        w = [np.ndarray(shape=(1, 1), dtype=float) for i in range(self.deep+1)]
        # self.da = self.db = self.dw = self.dz = self.a = self.z = self.b = self.w = w
        
        self.da = w.copy()
        self.w = w.copy()
        self.db = w.copy()
        self.dw = w.copy()
        self.dz = w.copy()
        self.a = w.copy()
        self.z = w.copy()
        self.b = w.copy()

        self.p = 0.5
        
        # 初始化w,b
        
        for idx in range(len(self.size_l)):
            self.w[idx + 1] = np.random.randn(
                self.size_l[idx][1], self.size_l[idx][0]) * 0.1
            # print("xx", self.size_l[idx][1], self.size_l[idx][0])
            # print(self.w[idx+1].shape)
            self.b[idx + 1] = np.random.randn(self.size_l[idx][1], 1) * 0.1
        # for idx in range(len(self.size_l)):
            # print(self.w[idx+1].shape)


    def linear(self, m_pre, m, ind):
        # for idx in range(len(self.size_l)):
        #     print(self.w[idx+1].shape)
        a_pre = self.a[ind - 1]
        w = self.w[ind]
        b = self.b[ind]
        # print(w.shape)
        # print(a_pre.shape)
        z = np.dot(w, a_pre) + b
        # print(w.shape, a_pre.shape, z.shape)
        return z

    def forward(self, x):
        # x = self.layers(x)
        self.a[0] = x
        for i in range(len(self.size_l)):
            m_pre = self.size_l[i][0]
            m = self.size_l[i][1]
            self.z[i+1] = self.linear(m_pre, m, i + 1)
            if i == len(self.size_l) - 1:
                self.a[i+1] = sigmoid(self.z[i+1])
            else:
                self.a[i+1] = relu(self.z[i+1])
                U = (np.random.rand(*self.a[i+1].shape) < self.p)
                self.a[i+1] *= U
                self.a[i+1] += 1e-10

            # print(self.z[i+1].shape)
        return

    def backward(self, Y, s):
        L = len(self.size_l)
        a, z, w = self.a, self.z, self.w
        # print(a[L])
        # print(a[L].shape)
        # a[L].reshape(-1)

        # print(a[L].T.shape)
        # print(Y.shape)
        # print("-------------")
        self.da[L] = -np.divide(Y, a[L][0]) + np.divide(1 - Y, 1 - a[L][0])
        # print(self.da[L].shape)
        # # back_activate =
        
        # print(self.da[L].shape)

        if s == 'relu':
            back_activate = relu_backward
        else:
            back_activate = sigmoid_back

        
        for i in reversed(range(1, L + 1)):
            if i == L:
                self.dz[i] = back_activate(self.da[i], z[i])
            else:
                self.dz[i] = relu_backward(self.da[i], z[i])

            # print(i)
            # print(self.dz[i].shape, a[i-1].T.shape)

            self.dw[i] = np.dot(self.dz[i], a[i-1].T)
            self.db[i] = np.sum(self.dz[i])
            # print(w[i].shape)
            # print(self.dz[i].shape)
            self.da[i-1] = np.dot(w[i].T, self.dz[i])
            # print(self.dw[i])
            # print(self.db[i])

    def update_argument(self):
        L = len(self.size_l)
        dw = self.dw
        db = self.db
        for idx in range(1, L + 1):
            self.w[idx] -= self.learning_rate * dw[idx]
            self.b[idx] -= self.learning_rate * db[idx]
            # print("dw", dw[idx][0:10])

    def cal_loss(self, y):
        y_pred = self.a[self.deep]
        # print(y_pred)
        loss = (-1.0 / y.shape[0]) * np.sum(y * np.log(y_pred + 1e-10) + (
            1 - y) * np.log(1 - y_pred + 1e-10))
        sum = 0
        # print(y_pred[0][1])
        # print(y.shape)
        # print(y_pred)
        for i in range(y.shape[0]):
            if abs(y[i] - y_pred[0][i]) <= 0.5:
                sum += 1
        print("acc: ", sum / y.shape[0])
        print("-------------", loss)

    def train(self, x, Y):
        self.forward(x)
        self.cal_loss(Y)
        self.backward(Y, 'sigmoid')
        self.update_argument()


#### some arugement

In [117]:
epoch = 2000
Learning_rate = 1e-3
Input_dim = 7
# np.random.seed(10)


In [118]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
114,0.0,1.0,1.0,0.208344,0.000,0.000000,0.028221,0.5
139,0.0,0.0,0.0,0.296306,0.000,0.000000,0.154588,0.5
436,0.0,1.0,1.0,0.258608,0.250,0.333333,0.067096,0.0
556,1.0,0.0,1.0,0.597889,0.125,0.000000,0.077294,0.5
745,0.0,0.0,0.0,0.874340,0.125,0.166667,0.138583,0.0


#### train

In [119]:
my_model = model(input_dim = Input_dim, learning_rate = Learning_rate)
for idx in range(epoch):
    # 挑选一些数据
    train_y = train_df['Survived']
    train_x = train_df.drop(['Survived'], axis=1)
    tr_x = train_x.to_numpy()
    tr_y = train_y.to_numpy()
    # print(tr_x.shape)
    # print(tr_y.shape)
    # for j in range(tr_x.shape[0]):
    model.train(my_model, tr_x.T, tr_y)
    


acc:  0.613603473227207
------------- 0.6785196826474384
acc:  0.613603473227207
------------- 0.674962176953168
acc:  0.613603473227207
------------- 0.6725132382155989
acc:  0.613603473227207
------------- 0.6710111571069577
acc:  0.613603473227207
------------- 0.6696743634647779
acc:  0.613603473227207
------------- 0.6691586305392863
acc:  0.613603473227207
------------- 0.6683347640139736
acc:  0.613603473227207
------------- 0.6679790403420606
acc:  0.613603473227207
------------- 0.6676344252107759
acc:  0.613603473227207
------------- 0.6674171381591963
acc:  0.613603473227207
------------- 0.667307200442226
acc:  0.613603473227207
------------- 0.6672354729174165
acc:  0.613603473227207
------------- 0.6672841791895501
acc:  0.613603473227207
------------- 0.6671847850406317
acc:  0.613603473227207
------------- 0.6670840588179244
acc:  0.613603473227207
------------- 0.6671655240074031
acc:  0.613603473227207
------------- 0.6673021030345575
acc:  0.613603473227207
---------